## Dependencies
* openai

In [1]:
import sys
sys.path.append("../")

In [2]:
import openai
import re
from src.decorators import load_key_secret

In [3]:
path = "/home/propietari/Documents/claus/"
filename = "OpenAI_keys_JordiPlanas.json"
@load_key_secret(path, filename, "secret")
def return_secret(secret):
    return secret

def create_completion(text_to_complete, number_of_samples, tokens):
    return openai.Completion.create(
        engine = "davinci-instruct-beta",
        prompt = text_to_complete,
        temperature = 0.9,
        max_tokens = tokens,
#         stop = "\n\n",
        n = number_of_samples #by default it gives you 4 choices so you multiply 4 times n
    )
def import_text_from_file(path, filename):
    file = path + filename
    with open(file) as f:
        lines = f.readlines()
    return lines

def calculate_tokens(text):
    string = ""
    for item in text:
        string = string + item
    return(len(string)/4)

find_digit = re.compile(r'\d')
def filter_response(response, stanza_length, max_verse_length, min_verse_length):
    clean_list = []
    for item in response["choices"]:
        new_text = item["text"]
        new_text = new_text.strip("\n")
        new_text = new_text.replace("\n\n", "\n")
        text_list = new_text.split("\n")
        string = "\n"
        flag = True
        if len(text_list) >= stanza_length:
            for verse in text_list[0:stanza_length]:
                if re.search(r'\d+', verse) is None and verse not in string and len(verse) < max_verse_length and len(verse) > min_verse_length:
                    string = string + verse + "\n"
                else:
                    flag = False
            if flag is True:        
                clean_list.append(string)
    return clean_list
            
    

def show_responses(base_text, filtered_response):
    string = ""
    for item in base_text:
        string = string + item
    i = 0
    for stanza in filtered_response:
        print(f"*** {i} ***\n{string}{stanza}\n")
        i += 1

In [4]:
openai.api_key = return_secret()

In [36]:
path = "../input/"
filename = "Song.txt"
original_text = import_text_from_file(path, filename)

maxim = 0
minim = 50
for i,item in enumerate(original_text):
    if len(item) > maxim:
        maxim = len(item)
    if len(item) < minim:
        minim = len(item)
    
print("max verse length is:", maxim, "and min verse length is:", minim)

max verse length is: 35 and min verse length is: 30


In [39]:
input_stanza_length = 4
output_stanza_length = 4
max_verse_length = 55
min_verse_length = 20
text_to_complete = original_text[0:input_stanza_length]
response = create_completion(text_to_complete, 20, int(calculate_tokens(text_to_complete) * 3))

In [40]:

show_responses(text_to_complete, filter_response(response, output_stanza_length, max_verse_length, min_verse_length))

*** 0 ***
I feel the wires, I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse
I keep discovering new things

You won't find us waving from the back rows
No, we're on the stage, don't try to shake our hands
We're not your typical ones
So reach for the stars


*** 1 ***
I feel the wires, I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse
I keep discovering new things

And I breathe and I see
And I touch but I can't feel
And I weight but I can't tell
And I form but I can't stay


*** 2 ***
I feel the wires, I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse
I keep discovering new things

There is no light without darkness
There is no dark without light
I shall not give up, I shall push on
I will not yield, I will persevere


*** 3 ***
I feel the wires, I feel the wires
My blood flows and my heart beats
I fear the lapse, I fear the lapse
I keep discovering new things

I break down and 

In [ ]:
summary_text